In [1]:
"""
https://watlab-blog.com/2022/10/08/2d-diffusion/
https://watlab-blog.com/2022/10/02/2d-advection/
https://python.keicode.com/lang/regular-expression-finditer.php
"""

'\nhttps://watlab-blog.com/2022/10/08/2d-diffusion/\nhttps://watlab-blog.com/2022/10/02/2d-advection/\nhttps://python.keicode.com/lang/regular-expression-finditer.php\n'

In [2]:
"""
更新予定
メリステムの境界条件を柔軟に決定できるようにする
  境界条件を2つ設定．既存のシミュレーション境界とメリステム境界

  時間変化及び，要素の値によって動的にメリステム境界を変化させたい．
  -> 境界外側を0で固定するのではなく，各ステップで内外の判定条件を作成する
  境界が内側に移動する場合と外側に内側に変化する場合で，分岐をさせると，自然ではない．
  よって，要素が0の場合には，内側の座標の値をコピーする．
  その後に境界外側の要素を0にする処理とする．


メリステム境界の判定と，メリステム境界の移動は異なる関数とする．
移動関数の後に，判定関数？

判定関数について
  どの要素を基準に境界を変化させるかについて，未定
  そこで，時間変化，力学的変化，要素変化など，後からパラメータを追加できるようにする．

  妥当だと考えられる，簡単な式で記述したい

    各要素の大きさ(最大値 - wusp)と，メリステムにおける相対的な位置
    但し，PZで分裂しにくい事を含めると，複雑となる．

    とりあえず，メリステム中心からのユークリッド距離を求めて，それを重みとして含める事とする．


更新箇所




今後


楕円の外の黄色線不要
各seedpointsに対応させて，面積を求める．
この面積はステップごとに分岐をせまられる．
  2倍より大きい　->　分裂    or   大きくする．



細胞分裂の再現 (動的な変化)
時間変化に対して，細胞の大きさが変化する(何をindexとするか?)
細胞の面積を近似計算によって求める．
面積が一定の大きさ(e.x. 2倍)になると分裂させる．

---外側は境界条件，固定されている---

外側も動的に変化させたい．


再考及びtask

微分方程式の離散化について，正当性の検証
離散式の記述を，分かりやすく

単にwus0を前の状態として入れてあるが、よいか？

楕円の境界条件，単にある点が楕円内にあるかどうかで判断

"""

'\n更新予定\nメリステムの境界条件を柔軟に決定できるようにする\n  境界条件を2つ設定．既存のシミュレーション境界とメリステム境界\n\n  時間変化及び，要素の値によって動的にメリステム境界を変化させたい．\n  -> 境界外側を0で固定するのではなく，各ステップで内外の判定条件を作成する\n  境界が内側に移動する場合と外側に内側に変化する場合で，分岐をさせると，自然ではない．\n  よって，要素が0の場合には，内側の座標の値をコピーする．\n  その後に境界外側の要素を0にする処理とする．\n\n\nメリステム境界の判定と，メリステム境界の移動は異なる関数とする．\n移動関数の後に，判定関数？\n\n判定関数について\n  どの要素を基準に境界を変化させるかについて，未定\n  そこで，時間変化，力学的変化，要素変化など，後からパラメータを追加できるようにする．\n\n  妥当だと考えられる，簡単な式で記述したい\n\n    各要素の大きさ(最大値 - wusp)と，メリステムにおける相対的な位置\n    但し，PZで分裂しにくい事を含めると，複雑となる．\n\n    とりあえず，メリステム中心からのユークリッド距離を求めて，それを重みとして含める事とする．\n\n\n更新箇所\n\n\n\n\n今後\n\n\n楕円の外の黄色線不要\n各seedpointsに対応させて，面積を求める．\nこの面積はステップごとに分岐をせまられる．\n  2倍より大きい\u3000->\u3000分裂    or   大きくする．\n\n\n\n細胞分裂の再現 (動的な変化)\n時間変化に対して，細胞の大きさが変化する(何をindexとするか?)\n細胞の面積を近似計算によって求める．\n面積が一定の大きさ(e.x. 2倍)になると分裂させる．\n\n---外側は境界条件，固定されている---\n\n外側も動的に変化させたい．\n\n\n再考及びtask\n\n微分方程式の離散化について，正当性の検証\n離散式の記述を，分かりやすく\n\n単にwus0を前の状態として入れてあるが、よいか？\n\n楕円の境界条件，単にある点が楕円内にあるかどうかで判断\n\n'

In [3]:
import numpy as np
import os
import glob
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
from matplotlib.path import Path
from matplotlib.animation import FuncAnimation
import re
import pandas as pd
from PIL import Image, ImageDraw
import io
import IPython.display as display
from collections import defaultdict
import sys
from scipy.spatial import Voronoi, voronoi_plot_2d
from sklearn.cluster import KMeans
from numba import jit
import cProfile
import pstats

from google.colab import files

In [5]:
# csvからparam取得

from google.colab import drive
drive.mount('/content/drive')
Sim_data= pd.read_csv('drive/My Drive/Colab Notebooks/Diffusion-ish/Mesh-type/Sim_Param.csv')


# Parameter_original
abc_idx, abc_par = Sim_data[1:1], Sim_data[0:1]

param_name = []
for n in abc_idx:
    param_name.append(n[0:4])

# pd 1row
param_ori = list(Sim_data.iloc[0])

for a, b in zip(param_name, param_ori):
    a = b

ID, K_CW, n_CW, k_Cr, K_CL, n_CL, K_CH, n_CH, k_Cp, b_Cr, b_Wr, k_WL, k_WC, k_Wr, K_WW, n_WW, K_WL, n_WL, K_WC, n_WC, k_Wp, b_Ww, b_Wr, ki_W, a_Cc, K_HW, n_HW, b_pp, k_pp, b_Hh, k_Hh, k_Ll, b_Ll, D_Ww, D_Cc, D_Ll = [a for a in param_ori]


Mounted at /content/drive


In [6]:
# Initial
wr_sigma11, wr_sigma22, wr_sigma12, wr_sigma21 = 900, 300, 0, 0
wp_sigma11, wp_sigma22, wp_sigma12, wp_sigma21 = 900, 300, 0, 0
cr_sigma11, cr_sigma22, cr_sigma12, cr_sigma21 = 200, 150, 0, 0
cp_sigma11, cp_sigma22, cp_sigma12, cp_sigma21 = 100, 50, 0, 0
eL_sigma11, eL_sigma22, eL_sigma12, eL_sigma21 = 60, 30, 0, 0
eR_sigma11, eR_sigma22, eR_sigma12, eR_sigma21 = 60, 30, 0, 0
h_sigma11, h_sigma22, h_sigma12, h_sigma21 = 10, 10, 0, 0

#            wr    wp    cr    cp    eL    eR     h
sigma11 = [    9,    9,    5,    5,    6,     6,    1]
sigma22 = [    3,    3,  1.5,  1.5,    3,     3,    1]
sigma12 = [    0,    0,    0,    0,    0,     0,    0]
sigma21 = [    0,    0,    0,    0,    0,     0,    0]


# 転置考慮　　原点を左上とする
mu_wusr_x, mu_wusr_y = 7.5, 2.5
mu_wusp_x, mu_wusp_y = 7.5, 2.5
mu_clvr_x, mu_clvr_y = 7.5, 7.5
mu_clvp_x, mu_clvp_y = 7.5, 7.5
mu_erfL_x, mu_erfL_y = 4, 5
mu_erfR_x, mu_erfR_y = 11, 5
mu_ham_x, mu_ham_y = 7.5, 2.5

element_str = ["wusr", "wusp", "clvr", "clvp", "erfL", "erfR", "ham"]

# Space
x_size, y_size = 15, 10
x_max, y_max = x_size, y_size  # range
dx, dy = 0.1, 0.1  # divide

# 楕円のパラメータ設定
ellip_long = 7  # 楕円の長軸の半径
ellip_short = 6  # 楕円の短軸の半径
ellip_x = 7.5  # 楕円の中心 x 座標
ellip_y = 3  # 楕円の中心 y 座標

global_seed_points = None

# Amplifier
wr_AMP, wp_AMP, cr_AMP, cp_AMP, eL_AMP, eR_AMP, h_AMP = 600, 150, 300, 150, 300, 300, 10

a = 1   # diffusion rate

# Calculation condition
dt = 0.0005  # Time
step = 10
result_interval = 5


#plsv_wusr, plsv_wusp, plsv_clvr, plsv_clvp, plsv_erfL, plsv_erfR, plsv_ham = 0, 0, 0, 0, 0, 0, 0
plsv_wusr, plsv_wusp, plsv_clvr, plsv_clvp, plsv_erfL, plsv_erfR, plsv_ham = 1, 1, 1, 1, 1, 1, 1

# component
wusr_str, wusp_str, clvr_str, clvp_str, erfL_str, erfR_str, ham_str = 'wusr', 'wusp', 'clvr', 'clvp', 'epflL', 'epflR', 'ham'
c_wusr, c_wusp, c_clvr, c_clvp, c_erfL, c_erfR, c_ham = 'jet', 'seismic', 'twilight_shifted', 'CMRmap', 'cividis', 'plasma', 'terrain'
# , , , , 'gist_stern', 'gnuplot',,, 'brg''gnuplot2' 'CMRmap''cubehelix'

#引数list
pull_sol = [dt, dx, dy, a, step, result_interval]


# GIF用
# 辞書 keylist
keys_name = ["wusr", "wusp", "clvr", "clvp", "erfL", "erfR", "ham"]

img_stock = defaultdict(list)

# 指定されたキーを追加
for key in keys_name:
    img_stock[key]  # defaultdictにキーを生成


In [7]:
def initial_field(x, y):
    # 各成分に対応するゼロ行列を生成
    zero_wusr, zero_wusp, zero_clvr, zero_clvp, zero_erfL, zero_erfR, zero_ham = [np.zeros((len(y), len(x))) for n in range(7)]

    # 2D Gaussian(sigma:分散共分散行列, mu:平均ベクトル)
    wr_sigma, wp_sigma, cr_sigma, cp_sigma, eL_sigma, eR_sigma, h_sigma = [
        np.array([[a, b], [c, d]]) for a, b, c, d in zip(sigma11, sigma12, sigma21, sigma22)
    ]

    # 各成分の行列式と逆行列を計算
    wr_det_sigma, wr_inv_sigma = np.linalg.det(wr_sigma), np.linalg.inv(wr_sigma)
    wp_det_sigma, wp_inv_sigma = np.linalg.det(wp_sigma), np.linalg.inv(wp_sigma)
    cr_det_sigma, cr_inv_sigma = np.linalg.det(cr_sigma), np.linalg.inv(cr_sigma)
    cp_det_sigma, cp_inv_sigma = np.linalg.det(cp_sigma), np.linalg.inv(cp_sigma)
    eL_det_sigma, eL_inv_sigma = np.linalg.det(eL_sigma), np.linalg.inv(eL_sigma)
    eR_det_sigma, eR_inv_sigma = np.linalg.det(eR_sigma), np.linalg.inv(eR_sigma)
    h_det_sigma, h_inv_sigma = np.linalg.det(h_sigma), np.linalg.inv(h_sigma)

    # Gaussian分布を計算する関数
    def get_arr(z, xtemp, ytemp, det_sigma, inv_sigma, AMPP):
        X, Y = np.meshgrid(x, y)
        pos = np.dstack((X, Y))
        arr = np.array([xtemp, ytemp])

        Z = (1 / (2 * np.pi * np.sqrt(det_sigma))) * np.exp(
            -0.5 * np.einsum('...k,kl,...l->...', pos - arr, inv_sigma, pos - arr)
        ) * AMPP
        return Z

    # 各変数のリストを作成
    zero_list = [zero_wusr, zero_wusp, zero_clvr, zero_clvp, zero_erfL, zero_erfR, zero_ham]
    mu_x_list = [mu_wusr_x, mu_wusp_x, mu_clvr_x, mu_clvp_x, mu_erfL_x, mu_erfR_x, mu_ham_x]
    mu_y_list = [mu_wusr_y, mu_wusp_y, mu_clvr_y, mu_clvp_y, mu_erfL_y, mu_erfR_y, mu_ham_y]
    det_list = [wr_det_sigma, wp_det_sigma, cr_det_sigma, cp_det_sigma, eL_det_sigma, eR_det_sigma, h_det_sigma]
    inv_list = [wr_inv_sigma, wp_inv_sigma, cr_inv_sigma, cp_inv_sigma, eL_inv_sigma, eR_inv_sigma, h_inv_sigma]
    AMP_list = [wr_AMP, wp_AMP, cr_AMP, cp_AMP, eL_AMP, eR_AMP, h_AMP]

    # 各成分のGaussian分布を計算
    zzz_wusr, zzz_wusp, zzz_clvr, zzz_clvp, zzz_erfL, zzz_erfR, zzz_ham = [
        get_arr(a, b, c, d, e, f) for a, b, c, d, e, f in zip(zero_list, mu_x_list, mu_y_list, det_list, inv_list, AMP_list)
    ]

    # 計算結果を返す
    return zzz_wusr, zzz_wusp, zzz_clvr, zzz_clvp, zzz_erfL, zzz_erfR, zzz_ham


def boundary_condition(a):
    a[:, 0], a[:, -1], a[0, :], a[-1, :] = 0, 0, 0, 0  # left, right, up. bottom
    return a


def meristem_boundary_mask(x, y, center_x, center_y, radius_func, element_values=None, threshold=None):

    """
    メリステムの境界マスクを作成する関数。
    radius_funcは時間や要素の値に応じて半径を返す関数。
    """
    X, Y = np.meshgrid(x, y)
    distance = np.sqrt((X - center_x)**2 + (Y - center_y)**2)
    radius = radius_func()
    mask = distance <= radius
    if element_values is not None and threshold is not None:
        mask = mask & (element_values >= threshold)
    return mask


def outer_boundary_mask(x, y, center_x, center_y, outer_radius_func):
    """
    外側の境界マスクを作成する関数。
    outer_radius_funcは時間や要素の値に応じて外側の半径を返す関数。
    """
    X, Y = np.meshgrid(x, y)
    distance = np.sqrt((X - center_x)**2 + (Y - center_y)**2)
    outer_radius = outer_radius_func()
    mask = distance <= outer_radius
    return mask


def radius_func(n):
    """
    時間ステップnに応じてメリステムの半径を計算する関数。
    例として、ステップごとに半径が0.01ずつ増加するモデル。
    """
    initial_radius = 3  # 初期半径（必要に応じて調整）
    expansion_rate = 0.01  # 半径の増加率
    return initial_radius + expansion_rate * n




In [8]:
def seed_points_maker(ellip_x, ellip_y, ellip_long, ellip_short, num_L1=30, num_L2=40, num_L3=50, shrink_factor=0.8):
    """
    楕円内のポイントを L1, L2, L3 層に分けて生成する
    L1, L2 cell の大きさが同じくらいになるように調整

    Args:
        ellip_x (float): 楕円の中心の x 座標。
        ellip_y (float): 楕円の中心の y 座標。
        ellip_long (float): 楕円の長軸の半径。
        ellip_short (float): 楕円の短軸の半径。
        num_L1 (int): L1 層のポイント数。
        num_L2 (int): L2 層のポイント数。
        num_L3 (int): L3 層のポイント数。

    Returns:
        list of dict: 各ポイントの座標と属性を含むリスト。
    """
    seedpoints = []

    # L1 層のポイント生成（楕円の境界線の内側）
    # 楕円の半径を調整して、シードポイントを内側に配置
    shift_factor_L1 = 0.9  # L1層の楕円の縮小率（調整可能）
    ellip_long_L1 = ellip_long * shift_factor_L1
    ellip_short_L1 = ellip_short * shift_factor_L1
    angles_L1 = np.linspace(0, 2 * np.pi, num_L1, endpoint=False)
    for angle in angles_L1:
        x = ellip_x + ellip_long_L1 * np.cos(angle)
        y = ellip_y + ellip_short_L1 * np.sin(angle)
        seedpoints.append({'x': x, 'y': y, 'layer': 'L1'})

    # L2 層のポイント生成（L1層の内側）
    shift_factor_L2 = 0.7  # L2層の楕円の縮小率
    ellip_long_L2 = ellip_long * shift_factor_L2
    ellip_short_L2 = ellip_short * shift_factor_L2
    angles_L2 = np.linspace(0, 2 * np.pi, num_L2, endpoint=False)
    for angle in angles_L2:
        x = ellip_x + ellip_long_L2 * np.cos(angle)
        y = ellip_y + ellip_short_L2 * np.sin(angle)
        seedpoints.append({'x': x, 'y': y, 'layer': 'L2'})

    # L3 層のポイント生成（L2 層の内側でランダムに配置）
    np.random.seed(42)  # 再現性のための乱数シード
    count = 0
    while count < num_L3:
        r = np.sqrt(np.random.uniform(0, 1)) * shift_factor_L2  # L2層より内側に配置
        theta = np.random.uniform(0, 2 * np.pi)
        x = ellip_x + ellip_long * r * np.cos(theta)
        y = ellip_y + ellip_short * r * np.sin(theta)
        seedpoints.append({'x': x, 'y': y, 'layer': 'L3'})
        count += 1

    return seedpoints

def update_seed_points(points, growth_rate=0.01):
    """
    シードポイントの座標を動的に更新する
    細胞の成長や分裂をシミュレートするために使用する
    """
    new_points = []
    for p in points:
        # 細胞の成長（位置の微小な変化を与える）
        dx = np.random.uniform(-growth_rate, growth_rate)
        dy = np.random.uniform(-growth_rate, growth_rate)
        new_x = p['x'] + dx
        new_y = p['y'] + dy
        new_points.append({'x': new_x, 'y': new_y, 'layer': p['layer']})
    return new_points

def calculate_cell_areas(vor, points):
    """
    各細胞（ボロノイ領域）の面積を計算する
    """
    areas = []
    for region_idx in vor.point_region:
        vertices = vor.regions[region_idx]
        if -1 in vertices or len(vertices) == 0:
            areas.append(0)
            continue
        # 頂点をNumPy配列に変換
        polygon = np.array([vor.vertices[i] for i in vertices])
        # 多角形を閉じるために最初の点を末尾に追加
        polygon = np.vstack((polygon, polygon[0]))
        # 面積を計算（Shoelace formula）
        area = 0.5 * np.abs(np.sum(polygon[:-1, 0] * polygon[1:, 1] - polygon[1:, 0] * polygon[:-1, 1]))
        areas.append(area)
    return areas


def divide_cells(points, areas, initial_areas, threshold=2.0):
    """
    細胞を分裂させる関数。
    面積が初期面積のthreshold倍以上になった細胞を分裂させる。
    """
    new_points = []
    new_initial_areas = []
    for i, area in enumerate(areas):
        if area >= threshold * initial_areas[i]:
            # 分裂処理：ポイントを2つに分ける
            point = points[i]
            delta = np.random.uniform(-0.1, 0.1, size=2)
            new_point1 = {'x': point['x'] + delta[0], 'y': point['y'] + delta[1], 'layer': point['layer']}
            new_point2 = {'x': point['x'] - delta[0], 'y': point['y'] - delta[1], 'layer': point['layer']}
            new_points.extend([new_point1, new_point2])
            # 初期面積も2つに分ける
            new_initial_areas.extend([initial_areas[i] / 2, initial_areas[i] / 2])
        else:
            new_points.append(points[i])
            new_initial_areas.append(initial_areas[i])

    return new_points, new_initial_areas



In [9]:
def divide_cells(points, areas, initial_areas, threshold=2.0, divide_active=True):
    """
    細胞を分裂させる関数。
    divide_active=Falseの場合、分裂を行わずシードポイントをそのまま返す。

    Args:
        points (list of dict): 現在のシードポイントのリスト。
        areas (list of float): 各シードポイントのボロノイ領域の面積。
        initial_areas (list of float): 各シードポイントの初期面積。
        threshold (float): 分裂の閾値（デフォルトは2.0）。
        divide_active (bool): 分裂を有効にするかどうか（デフォルトはTrue）。

    Returns:
        tuple: 更新後のシードポイントと初期面積のリスト。
    """
    if not divide_active:
        # 分裂を無効化：シードポイントと初期面積をそのまま返す
        return points, initial_areas

    new_points = []
    new_initial_areas = []
    for i, area in enumerate(areas):
        if area >= threshold * initial_areas[i]:
            # 分裂処理：ポイントを2つに分ける
            point = points[i]
            delta = np.random.uniform(-0.1, 0.1, size=2)
            new_point1 = {'x': point['x'] + delta[0], 'y': point['y'] + delta[1], 'layer': point['layer']}
            new_point2 = {'x': point['x'] - delta[0], 'y': point['y'] - delta[1], 'layer': point['layer']}
            new_points.extend([new_point1, new_point2])
            # 初期面積も2つに分ける
            new_initial_areas.extend([initial_areas[i] / 2, initial_areas[i] / 2])
        else:
            new_points.append(points[i])
            new_initial_areas.append(initial_areas[i])

    return new_points, new_initial_areas

@jit(nopython=True)
def update_values(wusr, wusp, clvr, clvp, erfL, erfR, ham, wusr0, wusp0, clvr0, clvp0, erfL0, erfR0, ham0,
                  dt, dx, dy, masks, mask, k_Wr, k_WL, K_WL, n_WL, k_WC, K_WC, n_WC, ki_W, K_WW, n_WW, b_Wr,
                  k_Wp, b_Ww, D_Ww, k_Cr, K_CL, n_CL, a_Cc, K_CW, n_CW, K_CH, n_CH, b_Cr, k_Cp, D_Cc,
                  k_Ll, b_Ll, D_Ll, k_pp, b_pp):

    """要素ごとの更新"""
    for i in range(1, wusr.shape[0] - 1):
        for j in range(1, wusr.shape[1] - 1):
            if mask[i, j] == 1:  # ボロノイ領域内の点のみ
                # 層ごとのパラメータ（例として拡散係数を層によって変更）
                # ここでは全て同じ値を使用してるが、必要に応じて変更する
                D_Ww_layer = D_Ww
                D_Cc_layer = D_Cc
                D_Ll_layer = D_Ll

                wusr[i, j] = wusr0[i, j] + dt * (
                    k_Wr * ((k_WL / (1 + ((erfL0[i, j] + erfR0[i, j]) / K_WL) ** n_WL)) +
                            (k_WC / (1 + ((clvp0[i, j]) / K_WC) ** n_WC))) *
                    (ki_W + ((wusp0[i, j] / K_WW) ** n_WW / (1 + (wusp0[i, j] / K_WW) ** n_WW))) - b_Wr * wusr0[i, j])

                wusp[i, j] = wusp0[i, j] + dt * (
                    k_Wp * wusr0[i, j] - b_Ww * wusp0[i, j] +
                    D_Ww_layer * ((wusp0[i + 1, j] - 2 * wusp0[i, j] + wusp0[i - 1, j]) / dx ** 2 +
                                  (wusp0[i, j + 1] - 2 * wusp0[i, j] + wusp0[i, j - 1]) / dy ** 2))

                clvr[i, j] = clvr0[i, j] + dt * (
                    (k_Cr / (1 + ((erfL0[i, j] + erfR0[i, j]) / K_CL) ** n_CL)) *
                    (a_Cc + ((wusp0[i, j] / K_CW) ** n_CW / (1 + (wusp0[i, j] / K_CW) ** n_CW))) *
                    (1 / (1 + (ham0[i, j] / K_CH) ** n_CH)) - b_Cr * clvr0[i, j])

                clvp[i, j] = clvp0[i, j] + dt * (
                    k_Cp * clvr0[i, j] - b_Cr * clvp0[i, j] +
                    D_Cc_layer * ((clvp0[i + 1, j] - 2 * clvp0[i, j] + clvp0[i - 1, j]) / dx ** 2 +
                                  (clvp0[i, j + 1] - 2 * clvp0[i, j] + clvp0[i, j - 1]) / dy ** 2))

                erfL[i, j] = erfL0[i, j] + dt * (
                    k_Ll - b_Ll * erfL0[i, j] +
                    D_Ll_layer * ((erfL0[i + 1, j] - 2 * erfL0[i, j] + erfL0[i - 1, j]) / dx ** 2 +
                                  (erfL0[i, j + 1] - 2 * erfL0[i, j] + erfL0[i, j - 1]) / dy ** 2))

                erfR[i, j] = erfR0[i, j] + dt * (
                    k_Ll - b_Ll * erfR0[i, j] +
                    D_Ll_layer * ((erfR0[i + 1, j] - 2 * erfR0[i, j] + erfR0[i - 1, j]) / dx ** 2 +
                                  (erfR0[i, j + 1] - 2 * erfR0[i, j] + erfR0[i, j - 1]) / dy ** 2))

                ham[i, j] = ham0[i, j] + dt * (k_pp - b_pp * ham0[i, j])

                found = True
                break  # マスクが見つかったのでループを抜ける


            else:
                # マスクに含まれない点をゼロに設定
                wusr[i, j] = 0
                wusp[i, j] = 0
                clvr[i, j] = 0
                clvp[i, j] = 0
                erfL[i, j] = 0
                erfR[i, j] = 0
                ham[i, j] = 0


def apply_boundary_conditions(values, mask):
    """
    境界条件を適用する関数。
    境界外の要素を0に設定し、必要に応じて内側の値をコピーする。
    """
    # 境界外の要素を0に設定
    values[~mask] = 0
    return values



# Error check用
def visualize_voronoi_and_mask(vor, masks, X, Y, step):
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))

    # ボロノイ領域のプロット
    voronoi_plot_2d(vor, ax=ax[0])
    ax[0].set_title(f'Voronoi Region - Step {step}')

    # マスクのプロット
    ax[1].imshow(np.sum(masks, axis=0), extent=(X.min(), X.max(), Y.min(), Y.max()), origin='lower')
    ax[1].set_title(f'Mask - Step {step}')

    plt.show()

# Errorcheck用
def check_mask_coverage(masks, X, Y):
    total_area = np.sum(masks) * (X[1] - X[0]) * (Y[1] - Y[0])
    print(f'Total masked area: {total_area}')




def sol_diffusion(x, y, wusr, wusp, clvr, clvp, erfL, erfR, ham,
                  pull, ID, points_list, layers, divide_active=True):
    dt, dx, dy, a, step, result_interval = pull

    # 初期面積の計算
    point_coords = np.array([[p['x'], p['y']] for p in points_list])
    vor = Voronoi(point_coords)
    initial_areas = calculate_cell_areas(vor, points_list)

    points = points_list  # シードポイントの初期化

    results = {
        'wusr': [], 'wusp': [], 'clvr': [], 'clvp': [], 'erfL': [], 'erfR': [], 'ham': [], 'n': []
    }

    for n in range(step):
        # シードポイントの更新
        points = update_seed_points(points)
        point_coords = np.array([[p['x'], p['y']] for p in points])
        point_layers = [p['layer'] for p in points]

        # ボロノイ分割の再計算
        vor = Voronoi(point_coords)

        # 面積の再計算
        areas = calculate_cell_areas(vor, points)

        # 細胞分裂の実行
        points, initial_areas = divide_cells(points, areas, initial_areas)

        print(f"  更新後のシードポイント数: {len(points)}")

        # マスクの再作成
        masks = np.zeros((len(points), len(y), len(x)))
        X, Y = np.meshgrid(x, y)
        for idx, region_idx in enumerate(vor.point_region):
            region = vor.regions[region_idx]
            if -1 in region or len(region) == 0:
                continue

            polygon = vor.vertices[region]
            # バウンディングボックスの計算
            min_x, min_y = np.min(polygon, axis=0)
            max_x, max_y = np.max(polygon, axis=0)
            # バウンディングボックス内のポイントのみチェック
            mask_subset = (X >= min_x) & (X <= max_x) & (Y >= min_y) & (Y <= max_y)
            subset_X = X[mask_subset]
            subset_Y = Y[mask_subset]
            if len(subset_X) == 0:
                continue
            grid_points = np.vstack((subset_X.ravel(), subset_Y.ravel())).T
            mask_cell = Path(polygon).contains_points(grid_points).reshape(-1)
            mask_full = np.zeros(X.shape, dtype=bool)
            mask_full[mask_subset] = mask_cell
            masks[idx] = mask_full



        # ここで細胞の面積を計算（将来的に使用）
        areas = calculate_cell_areas(vor, points)



        # メリステム境界マスクの作成
        current_radius = radius_func(n)
        mask = meristem_boundary_mask(x, y, ellip_x, ellip_y, lambda: current_radius)

        # 更新前の値をコピー
        wusr0, wusp0, clvr0, clvp0, erfL0, erfR0, ham0 = (
            wusr.copy(), wusp.copy(), clvr.copy(), clvp.copy(), erfL.copy(), erfR.copy(), ham.copy())

        # 物質の濃度更新
        update_values(wusr, wusp, clvr, clvp, erfL, erfR, ham,
                      wusr0, wusp0, clvr0, clvp0, erfL0, erfR0, ham0,
                      dt, dx, dy, masks, mask,
                      k_Wr, k_WL, K_WL, n_WL, k_WC, K_WC, n_WC, ki_W,
                      K_WW, n_WW, b_Wr, k_Wp, b_Ww, D_Ww,
                      k_Cr, K_CL, n_CL, a_Cc, K_CW, n_CW, K_CH, n_CH,
                      b_Cr, k_Cp, D_Cc, k_Ll, b_Ll, D_Ll, k_pp, b_pp)

        # 境界条件の適用
        wusr = apply_boundary_conditions(wusr, mask)
        wusp = apply_boundary_conditions(wusp, mask)
        clvr = apply_boundary_conditions(clvr, mask)
        clvp = apply_boundary_conditions(clvp, mask)
        erfL = apply_boundary_conditions(erfL, mask)
        erfR = apply_boundary_conditions(erfR, mask)
        ham = apply_boundary_conditions(ham, mask)

        if n % result_interval == 0:
            # 結果を保存
            results['wusr'].append(wusr.copy())
            results['wusp'].append(wusp.copy())
            results['clvr'].append(clvr.copy())
            results['clvp'].append(clvp.copy())
            results['erfL'].append(erfL.copy())
            results['erfR'].append(erfR.copy())
            results['ham'].append(ham.copy())
            results['n'].append(n)

    return results, point_coords, vor, point_layers




In [10]:
def plot_voronoi_with_layers(vor, points, layers, ellip_x, ellip_y, ellip_long, ellip_short):
    """層情報を持つボロノイ図の描画"""
    fig, ax = plt.subplots(figsize=(8, 8))
    voronoi_plot_2d(vor, ax=ax, show_vertices=False, line_colors='gray',
                    line_width=1, line_alpha=0.6, point_size=2)

    # 層ごとにポイントをプロット
    layer_colors = {'L1': 'red', 'L2': 'green', 'L3': 'blue'}
    for layer in ['L1', 'L2', 'L3']:
        idx = [i for i, l in enumerate(layers) if l == layer]
        ax.plot(points[idx, 0], points[idx, 1], 'o', label=layer, color=layer_colors[layer])

    # 楕円の描画(外側の境界線)
    theta = np.linspace(0, 2 * np.pi, 100)
    ellipse_x = ellip_x + ellip_long * np.cos(theta)
    ellipse_y = ellip_y + ellip_short * np.sin(theta)
    #ax.plot(ellipse_x, ellipse_y, 'k--', linewidth=1)


    # L1層とL2層の境界線（オプション）
    # L1層の境界線
    ellipse_L1_x = ellip_x + ellip_long * shift_factor_L1 * np.cos(theta)
    ellipse_L1_y = ellip_y + ellip_short * shift_factor_L1 * np.sin(theta)
    ax.plot(ellipse_L1_x, ellipse_L1_y, 'r--', linewidth=1)

    # L2層の境界線
    ellipse_L2_x = ellip_x + ellip_long * shift_factor_L2 * np.cos(theta)
    ellipse_L2_y = ellip_y + ellip_short * shift_factor_L2 * np.sin(theta)
    ax.plot(ellipse_L2_x, ellipse_L2_y, 'g--', linewidth=1)


    ax.set_xlim(0, x_size)
    ax.set_ylim(0, y_size)
    ax.legend()
    plt.title('Voronoi Diagram with Layers')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.grid(True)
    plt.show()

def plot(x, y, z, n, name, color, ID, vor=None, points=None, layers=None, ellip_x=None, ellip_y=None, ellip_long=None, ellip_short=None):
    plt.rcParams['font.size'] = 14
    plt.rcParams['xtick.direction'] = 'in'
    plt.rcParams['ytick.direction'] = 'in'

    fig, ax1 = plt.subplots(figsize=(x_size, y_size))
    ax1.yaxis.set_ticks_position('both')
    ax1.xaxis.set_ticks_position('both')
    ax1.set_xlabel('x')
    ax1.set_ylabel('y')

    # メインのデータの描画
    im = ax1.imshow(z.T, vmin=0, vmax=100, extent=[0, x_size, 0, y_size], aspect='equal', cmap=f'{color}', origin='lower')
    ax1.text(0.1, 0.1, f'{name}, Step={n}', color="black")
    ax1.text(7, 0.1, f'{name}, Step={n}', color="white")
    cbar = fig.colorbar(im)
    cbar.set_label(f'{name}')

    # ボロノイ分割とシードポイントの描画
    if vor is not None and points is not None:
        voronoi_plot_2d(vor, ax=ax1, show_vertices=False, line_colors='yellow', line_width=2, line_alpha=0.6, point_size=10)
        # 層ごとにポイントをプロット
        layer_colors = {'L1': 'red', 'L2': 'green', 'L3': 'blue'}
        for layer in ['L1', 'L2', 'L3']:
            idx = [i for i, l in enumerate(layers) if l == layer]
            ax1.plot(points[idx, 0], points[idx, 1], 'o', label=layer, color=layer_colors[layer])

        # 楕円の描画
        theta = np.linspace(0, 2 * np.pi, 100)
        ellipse_x = ellip_x + ellip_long * np.cos(theta)
        ellipse_y = ellip_y + ellip_short * np.sin(theta)
        ax1.plot(ellipse_x, ellipse_y, 'w--', linewidth=2)  # 楕円の境界を白に

    # x軸とy軸の範囲を設定
    ax1.set_xlim(0, x_size)
    ax1.set_ylim(0, y_size)
    ax1.legend()

    # メモリ上に保存
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    img_stock[name].append(Image.open(buf))

    plt.show()
    plt.close(fig)


In [11]:
def plot_diffusion(x, y, results, points, vor, layers, ellip_x, ellip_y, ellip_long, ellip_short, ID):
    n_list = results['n']
    for idx, n in enumerate(n_list):
        element_var = [
        results['wusr'][idx].T, results['wusp'][idx].T, results['clvr'][idx].T, results['clvp'][idx].T, results['erfL'][idx].T, results['erfR'][idx].T, results['ham'][idx].T
        ]

        for field, name, color in zip(
            element_var,
            ["wusr", "wusp", "clvr", "clvp", "erfL", "erfR", "ham"],
            [c_wusr, c_wusp, c_clvr, c_clvp, c_erfL, c_erfR, c_ham]
        ):
            if globals().get(f'plsv_{name}', True):
                plot(
                    x, y, field, n=n, name=name, color=color, ID=ID, vor=vor, points=points, layers=layers,
                    ellip_x=ellip_x, ellip_y=ellip_y,
                    ellip_long=ellip_long, ellip_short=ellip_short
                )

In [12]:
# Initial field
x, y = np.linspace(0, x_max, int(x_max / dx)), np.linspace(0, y_max, int(y_max / dy))
wusr, wusp, clvr, clvp, erfL, erfR, ham = initial_field(x, y)


# 境界条件を設定する
wusr = boundary_condition(wusr)
wusp = boundary_condition(wusp)
clvr = boundary_condition(clvr)
clvp = boundary_condition(clvp)
erfL = boundary_condition(erfL)
erfR = boundary_condition(erfR)
ham = boundary_condition(ham)


# 安定性の確認
nu_x, nu_y = a * dt / dx ** 2, a * dt / dy ** 2
print('nu_x, nu_y=', nu_x, nu_y)

# シードポイントの生成
points_list = seed_points_maker(
    ellip_x=ellip_x,
    ellip_y=ellip_y,
    ellip_long=ellip_long,
    ellip_short=ellip_short,
    num_L1=30,  # L1 層のポイント数（調整可能）
    num_L2=33,  # L2 層のポイント数（調整可能）
    num_L3=40   # L3 層のポイント数（調整可能）
)

# ポイントの座標と属性を取得
points = np.array([[p['x'], p['y']] for p in points_list])
layers = [p['layer'] for p in points_list]



# シミュレーションの実行時に分裂を無効化
divide_active = False  # 一時的に分裂を無効化

results, points, vor, layers = sol_diffusion(
    x, y, wusr, wusp, clvr, clvp, erfL, erfR, ham,
    pull_sol, ID, points_list, layers,
    divide_active=divide_active
)





# シミュレーションの実行
pull_sol = [dt, dx, dy, a, step, result_interval]
ID = 0
results, points, vor, layers = sol_diffusion(x, y, wusr, wusp, clvr, clvp, erfL, erfR, ham, pull_sol, ID, points_list, layers)


nu_x, nu_y= 0.04999999999999999 0.04999999999999999
  更新後のシードポイント数: 133
  更新後のシードポイント数: 193
  更新後のシードポイント数: 313
  更新後のシードポイント数: 553
  更新後のシードポイント数: 1033
  更新後のシードポイント数: 1993
  更新後のシードポイント数: 3913
  更新後のシードポイント数: 7753
  更新後のシードポイント数: 15433
  更新後のシードポイント数: 30793


KeyboardInterrupt: 

In [ ]:
plot_diffusion(x, y, results, points, vor, layers, ellip_x, ellip_y, ellip_long, ellip_short, ID)


In [13]:
import cProfile
import pstats
import io

pr = cProfile.Profile()
pr.enable()

# 実行する関数
results, points, vor, layers = sol_diffusion(x, y, wusr, wusp, clvr, clvp, erfL, erfR, ham, pull_sol, ID, points_list, layers)

pr.disable()
s = io.StringIO()
sortby = 'cumulative'
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats(10)  # 上位10個の関数を表示
print(s.getvalue())


  更新後のシードポイント数: 133
  更新後のシードポイント数: 193
  更新後のシードポイント数: 313
  更新後のシードポイント数: 553
  更新後のシードポイント数: 1033
  更新後のシードポイント数: 1993
  更新後のシードポイント数: 3913
  更新後のシードポイント数: 7753
  更新後のシードポイント数: 15433


KeyboardInterrupt: 

In [14]:
# -*- coding: utf-8 -*-
import numpy as np
import os
import glob
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
from matplotlib.path import Path
from matplotlib.animation import FuncAnimation
import re
import pandas as pd
from PIL import Image, ImageDraw
import io
import IPython.display as display
from collections import defaultdict
import sys
from scipy.spatial import Voronoi, voronoi_plot_2d
from sklearn.cluster import KMeans
from numba import jit, njit
import cProfile
import pstats

from google.colab import files
from google.colab import drive

# ドライブのマウント
drive.mount('/content/drive')
Sim_data= pd.read_csv('drive/My Drive/Colab Notebooks/Diffusion-ish/Mesh-type/Sim_Param.csv')

# Parameter_original
abc_idx, abc_par = Sim_data[1:1], Sim_data[0:1]

param_name = []
for n in abc_idx:
    param_name.append(n[0:4])

# pd 1row
param_ori = list(Sim_data.iloc[0])

for a, b in zip(param_name, param_ori):
    a = b

# 以下、各パラメータの定義
ID, K_CW, n_CW, k_Cr, K_CL, n_CL, K_CH, n_CH, k_Cp, b_Cr, b_Wr, k_WL, k_WC, k_Wr, K_WW, n_WW, K_WL, n_WL, K_WC, n_WC, k_Wp, b_Ww, b_Wr, ki_W, a_Cc, K_HW, n_HW, b_pp, k_pp, b_Hh, k_Hh, k_Ll, b_Ll, D_Ww, D_Cc, D_Ll = [a for a in param_ori]

# Initial
wr_sigma11, wr_sigma22, wr_sigma12, wr_sigma21 = 900, 300, 0, 0
wp_sigma11, wp_sigma22, wp_sigma12, wp_sigma21 = 900, 300, 0, 0
cr_sigma11, cr_sigma22, cr_sigma12, cr_sigma21 = 200, 150, 0, 0
cp_sigma11, cp_sigma22, cp_sigma12, cp_sigma21 = 100, 50, 0, 0
eL_sigma11, eL_sigma22, eL_sigma12, eL_sigma21 = 60, 30, 0, 0
eR_sigma11, eR_sigma22, eR_sigma12, eR_sigma21 = 60, 30, 0, 0
h_sigma11, h_sigma22, h_sigma12, h_sigma21 = 10, 10, 0, 0

#            wr    wp    cr    cp    eL    eR     h
sigma11 = [    9,    9,    5,    5,    6,     6,    1]
sigma22 = [    3,    3,  1.5,  1.5,    3,     3,    1]
sigma12 = [    0,    0,    0,    0,    0,     0,    0]
sigma21 = [    0,    0,    0,    0,    0,     0,    0]

# 転置考慮　　原点を左上とする
mu_wusr_x, mu_wusr_y = 7.5, 2.5
mu_wusp_x, mu_wusp_y = 7.5, 2.5
mu_clvr_x, mu_clvr_y = 7.5, 7.5
mu_clvp_x, mu_clvp_y = 7.5, 7.5
mu_erfL_x, mu_erfL_y = 4, 5
mu_erfR_x, mu_erfR_y = 11, 5
mu_ham_x, mu_ham_y = 7.5, 2.5

element_str = ["wusr", "wusp", "clvr", "clvp", "erfL", "erfR", "ham"]

# Space
x_size, y_size = 15, 10
x_max, y_max = x_size, y_size  # range
dx, dy = 0.1, 0.1  # divide

# 楕円のパラメータ設定
ellip_long = 7  # 楕円の長軸の半径
ellip_short = 6  # 楕円の短軸の半径
ellip_x = 7.5  # 楕円の中心 x 座標
ellip_y = 3  # 楕円の中心 y 座標

global_seed_points = None

# Amplifier
wr_AMP, wp_AMP, cr_AMP, cp_AMP, eL_AMP, eR_AMP, h_AMP = 600, 150, 300, 150, 300, 300, 10

a = 1   # diffusion rate

# Calculation condition
dt = 0.0005  # Time
step = 10
result_interval = 5

#plsv_wusr, plsv_wusp, plsv_clvr, plsv_clvp, plsv_erfL, plsv_erfR, plsv_ham = 0, 0, 0, 0, 0, 0, 0
plsv_wusr, plsv_wusp, plsv_clvr, plsv_clvp, plsv_erfL, plsv_erfR, plsv_ham = 1, 1, 1, 1, 1, 1, 1

# component
wusr_str, wusp_str, clvr_str, clvp_str, erfL_str, erfR_str, ham_str = 'wusr', 'wusp', 'clvr', 'clvp', 'epflL', 'epflR', 'ham'
c_wusr, c_wusp, c_clvr, c_clvp, c_erfL, c_erfR, c_ham = 'jet', 'seismic', 'twilight_shifted', 'CMRmap', 'cividis', 'plasma', 'terrain'
# , , , , 'gist_stern', 'gnuplot',,, 'brg''gnuplot2' 'CMRmap''cubehelix'

# 引数list
pull_sol = [dt, dx, dy, a, step, result_interval]

# GIF用
# 辞書 keylist
keys_name = ["wusr", "wusp", "clvr", "clvp", "erfL", "erfR", "ham"]

img_stock = defaultdict(list)

# 指定されたキーを追加
for key in keys_name:
    img_stock[key]  # defaultdictにキーを生成

def initial_field(x, y):
    # 各成分に対応するゼロ行列を生成
    zero_wusr, zero_wusp, zero_clvr, zero_clvp, zero_erfL, zero_erfR, zero_ham = [np.zeros((len(y), len(x))) for n in range(7)]

    # 2D Gaussian(sigma:分散共分散行列, mu:平均ベクトル)
    wr_sigma, wp_sigma, cr_sigma, cp_sigma, eL_sigma, eR_sigma, h_sigma = [
        np.array([[a, b], [c, d]]) for a, b, c, d in zip(sigma11, sigma12, sigma21, sigma22)
    ]

    # 各成分の行列式と逆行列を計算
    wr_det_sigma, wr_inv_sigma = np.linalg.det(wr_sigma), np.linalg.inv(wr_sigma)
    wp_det_sigma, wp_inv_sigma = np.linalg.det(wp_sigma), np.linalg.inv(wp_sigma)
    cr_det_sigma, cr_inv_sigma = np.linalg.det(cr_sigma), np.linalg.inv(cr_sigma)
    cp_det_sigma, cp_inv_sigma = np.linalg.det(cp_sigma), np.linalg.inv(cp_sigma)
    eL_det_sigma, eL_inv_sigma = np.linalg.det(eL_sigma), np.linalg.inv(eL_sigma)
    eR_det_sigma, eR_inv_sigma = np.linalg.det(eR_sigma), np.linalg.inv(eR_sigma)
    h_det_sigma, h_inv_sigma = np.linalg.det(h_sigma), np.linalg.inv(h_sigma)

    # Gaussian分布を計算する関数
    def get_arr(z, xtemp, ytemp, det_sigma, inv_sigma, AMPP):
        X, Y = np.meshgrid(x, y)
        pos = np.dstack((X, Y))
        arr = np.array([xtemp, ytemp])

        Z = (1 / (2 * np.pi * np.sqrt(det_sigma))) * np.exp(
            -0.5 * np.einsum('...k,kl,...l->...', pos - arr, inv_sigma, pos - arr)
        ) * AMPP
        return Z

    # 各変数のリストを作成
    zero_list = [zero_wusr, zero_wusp, zero_clvr, zero_clvp, zero_erfL, zero_erfR, zero_ham]
    mu_x_list = [mu_wusr_x, mu_wusp_x, mu_clvr_x, mu_clvp_x, mu_erfL_x, mu_erfR_x, mu_ham_x]
    mu_y_list = [mu_wusr_y, mu_wusp_y, mu_clvr_y, mu_clvp_y, mu_erfL_y, mu_erfR_y, mu_ham_y]
    det_list = [wr_det_sigma, wp_det_sigma, cr_det_sigma, cp_det_sigma, eL_det_sigma, eR_det_sigma, h_det_sigma]
    inv_list = [wr_inv_sigma, wp_inv_sigma, cr_inv_sigma, cp_inv_sigma, eL_inv_sigma, eR_inv_sigma, h_inv_sigma]
    AMP_list = [wr_AMP, wp_AMP, cr_AMP, cp_AMP, eL_AMP, eR_AMP, h_AMP]

    # 各成分のGaussian分布を計算
    zzz_wusr, zzz_wusp, zzz_clvr, zzz_clvp, zzz_erfL, zzz_erfR, zzz_ham = [
        get_arr(a, b, c, d, e, f) for a, b, c, d, e, f in zip(zero_list, mu_x_list, mu_y_list, det_list, inv_list, AMP_list)
    ]

    # 計算結果を返す
    return zzz_wusr, zzz_wusp, zzz_clvr, zzz_clvp, zzz_erfL, zzz_erfR, zzz_ham

def boundary_condition(a):
    a[:, 0], a[:, -1], a[0, :], a[-1, :] = 0, 0, 0, 0  # left, right, up, bottom
    return a

def meristem_boundary_mask(x, y, center_x, center_y, radius_func, element_values=None, threshold=None):
    """
    メリステムの境界マスクを作成する関数。
    radius_funcは時間や要素の値に応じて半径を返す関数。
    """
    X, Y = np.meshgrid(x, y)
    distance = np.sqrt((X - center_x)**2 + (Y - center_y)**2)
    radius = radius_func()
    mask = distance <= radius
    if element_values is not None and threshold is not None:
        mask = mask & (element_values >= threshold)
    return mask

def outer_boundary_mask(x, y, center_x, center_y, outer_radius_func):
    """
    外側の境界マスクを作成する関数。
    outer_radius_funcは時間や要素の値に応じて外側の半径を返す関数。
    """
    X, Y = np.meshgrid(x, y)
    distance = np.sqrt((X - center_x)**2 + (Y - center_y)**2)
    outer_radius = outer_radius_func()
    mask = distance <= outer_radius
    return mask

def radius_func(n):
    """
    時間ステップnに応じてメリステムの半径を計算する関数。
    例として、ステップごとに半径が0.01ずつ増加するモデル。
    """
    initial_radius = 3  # 初期半径（必要に応じて調整）
    expansion_rate = 0.01  # 半径の増加率
    return initial_radius + expansion_rate * n

def seed_points_maker(ellip_x, ellip_y, ellip_long, ellip_short, num_L1=30, num_L2=40, num_L3=50, shrink_factor=0.8):
    """
    楕円内のポイントを L1, L2, L3 層に分けて生成する
    L1, L2 cell の大きさが同じくらいになるように調整

    Args:
        ellip_x (float): 楕円の中心の x 座標。
        ellip_y (float): 楕円の中心の y 座標。
        ellip_long (float): 楕円の長軸の半径。
        ellip_short (float): 楕円の短軸の半径。
        num_L1 (int): L1 層のポイント数。
        num_L2 (int): L2 層のポイント数。
        num_L3 (int): L3 層のポイント数。

    Returns:
        list of dict: 各ポイントの座標と属性を含むリスト。
    """
    seedpoints = []

    # L1 層のポイント生成（楕円の境界線の内側）
    shift_factor_L1 = 0.9  # L1層の楕円の縮小率（調整可能）
    ellip_long_L1 = ellip_long * shift_factor_L1
    ellip_short_L1 = ellip_short * shift_factor_L1
    angles_L1 = np.linspace(0, 2 * np.pi, num_L1, endpoint=False)
    for angle in angles_L1:
        x = ellip_x + ellip_long_L1 * np.cos(angle)
        y = ellip_y + ellip_short_L1 * np.sin(angle)
        seedpoints.append({'x': x, 'y': y, 'layer': 'L1'})

    # L2 層のポイント生成（L1層の内側）
    shift_factor_L2 = 0.7  # L2層の楕円の縮小率
    ellip_long_L2 = ellip_long * shift_factor_L2
    ellip_short_L2 = ellip_short * shift_factor_L2
    angles_L2 = np.linspace(0, 2 * np.pi, num_L2, endpoint=False)
    for angle in angles_L2:
        x = ellip_x + ellip_long_L2 * np.cos(angle)
        y = ellip_y + ellip_short_L2 * np.sin(angle)
        seedpoints.append({'x': x, 'y': y, 'layer': 'L2'})

    # L3 層のポイント生成（L2 層の内側でランダムに配置）
    np.random.seed(42)  # 再現性のための乱数シード
    count = 0
    while count < num_L3:
        r = np.sqrt(np.random.uniform(0, 1)) * shift_factor_L2  # L2層より内側に配置
        theta = np.random.uniform(0, 2 * np.pi)
        x = ellip_x + ellip_long * r * np.cos(theta)
        y = ellip_y + ellip_short * r * np.sin(theta)
        seedpoints.append({'x': x, 'y': y, 'layer': 'L3'})
        count += 1

    return seedpoints

def update_seed_points(points, growth_rate=0.01):
    """
    シードポイントの座標を動的に更新する
    細胞の成長や分裂をシミュレートするために使用する
    """
    new_points = []
    for p in points:
        # 細胞の成長（位置の微小な変化を与える）
        dx = np.random.uniform(-growth_rate, growth_rate)
        dy = np.random.uniform(-growth_rate, growth_rate)
        new_x = p['x'] + dx
        new_y = p['y'] + dy
        new_points.append({'x': new_x, 'y': new_y, 'layer': p['layer']})
    return new_points

def calculate_cell_areas(vor, points):
    """
    各細胞（ボロノイ領域）の面積を計算する
    """
    areas = []
    for region_idx in vor.point_region:
        vertices = vor.regions[region_idx]
        if -1 in vertices or len(vertices) == 0:
            areas.append(0)
            continue
        # 頂点をNumPy配列に変換
        polygon = np.array([vor.vertices[i] for i in vertices])
        # 多角形を閉じるために最初の点を末尾に追加
        polygon = np.vstack((polygon, polygon[0]))
        # 面積を計算（Shoelace formula）
        area = 0.5 * np.abs(np.sum(polygon[:-1, 0] * polygon[1:, 1] - polygon[1:, 0] * polygon[:-1, 1]))
        areas.append(area)
    return areas

def divide_cells(points, areas, initial_areas, threshold=2.0, divide_active=True):
    """
    細胞を分裂させる関数。
    divide_active=Falseの場合、分裂を行わずシードポイントをそのまま返す。

    Args:
        points (list of dict): 現在のシードポイントのリスト。
        areas (list of float): 各シードポイントのボロノイ領域の面積。
        initial_areas (list of float): 各シードポイントの初期面積。
        threshold (float): 分裂の閾値（デフォルトは2.0）。
        divide_active (bool): 分裂を有効にするかどうか（デフォルトはTrue）。

    Returns:
        tuple: 更新後のシードポイントと初期面積のリスト。
    """
    if not divide_active:
        print("細胞分裂が無効化されています。")
        return points, initial_areas

    new_points = []
    new_initial_areas = []
    for i, area in enumerate(areas):
        if area >= threshold * initial_areas[i]:
            # 分裂処理：ポイントを2つに分ける
            print(f"細胞 {i} が分裂します。")
            point = points[i]
            delta = np.random.uniform(-0.1, 0.1, size=2)
            new_point1 = {'x': point['x'] + delta[0], 'y': point['y'] + delta[1], 'layer': point['layer']}
            new_point2 = {'x': point['x'] - delta[0], 'y': point['y'] - delta[1], 'layer': point['layer']}
            new_points.extend([new_point1, new_point2])
            # 初期面積も2つに分ける
            new_initial_areas.extend([initial_areas[i] / 2, initial_areas[i] / 2])
        else:
            new_points.append(points[i])
            new_initial_areas.append(initial_areas[i])

    return new_points, new_initial_areas

@njit
def update_values(wusr, wusp, clvr, clvp, erfL, erfR, ham, wusr0, wusp0, clvr0, clvp0, erfL0, erfR0, ham0,
                  dt, dx, dy, masks, mask, k_Wr, k_WL, K_WL, n_WL, k_WC, K_WC, n_WC, ki_W, K_WW, n_WW, b_Wr,
                  k_Wp, b_Ww, D_Ww, k_Cr, K_CL, n_CL, a_Cc, K_CW, n_CW, K_CH, n_CH, b_Cr, k_Cp, D_Cc,
                  k_Ll, b_Ll, D_Ll, k_pp, b_pp):
    """要素ごとの更新"""
    for i in range(1, wusr.shape[0] - 1):
        for j in range(1, wusr.shape[1] - 1):
            if mask[i, j]:
                # 計算処理
                wusr[i, j] = wusr0[i, j] + dt * (
                    k_Wr * ((k_WL / (1 + ((erfL0[i, j] + erfR0[i, j]) / K_WL) ** n_WL)) +
                            (k_WC / (1 + ((clvp0[i, j]) / K_WC) ** n_WC))) *
                    (ki_W + ((wusp0[i, j] / K_WW) ** n_WW / (1 + (wusp0[i, j] / K_WW) ** n_WW))) - b_Wr * wusr0[i, j])

                wusp[i, j] = wusp0[i, j] + dt * (
                    k_Wp * wusr0[i, j] - b_Ww * wusp0[i, j] +
                    D_Ww * ((wusp0[i + 1, j] - 2 * wusp0[i, j] + wusp0[i - 1, j]) / dx ** 2 +
                           (wusp0[i, j + 1] - 2 * wusp0[i, j] + wusp0[i, j - 1]) / dy ** 2))

                clvr[i, j] = clvr0[i, j] + dt * (
                    (k_Cr / (1 + ((erfL0[i, j] + erfR0[i, j]) / K_CL) ** n_CL)) *
                    (a_Cc + ((wusp0[i, j] / K_CW) ** n_CW / (1 + (wusp0[i, j] / K_CW) ** n_CW))) *
                    (1 / (1 + (ham0[i, j] / K_CH) ** n_CH)) - b_Cr * clvr0[i, j])

                clvp[i, j] = clvp0[i, j] + dt * (
                    k_Cp * clvr0[i, j] - b_Cr * clvp0[i, j] +
                    D_Cc * ((clvp0[i + 1, j] - 2 * clvp0[i, j] + clvp0[i - 1, j]) / dx ** 2 +
                           (clvp0[i, j + 1] - 2 * clvp0[i, j] + clvp0[i, j - 1]) / dy ** 2))

                erfL[i, j] = erfL0[i, j] + dt * (
                    k_Ll - b_Ll * erfL0[i, j] +
                    D_Ll * ((erfL0[i + 1, j] - 2 * erfL0[i, j] + erfL0[i - 1, j]) / dx ** 2 +
                           (erfL0[i, j + 1] - 2 * erfL0[i, j] + erfL0[i, j - 1]) / dy ** 2))

                erfR[i, j] = erfR0[i, j] + dt * (
                    k_Ll - b_Ll * erfR0[i, j] +
                    D_Ll * ((erfR0[i + 1, j] - 2 * erfR0[i, j] + erfR0[i - 1, j]) / dx ** 2 +
                           (erfR0[i, j + 1] - 2 * erfR0[i, j] + erfR0[i, j - 1]) / dy ** 2))

                ham[i, j] = ham0[i, j] + dt * (k_pp - b_pp * ham0[i, j])
            else:
                # マスクに含まれない点をゼロに設定
                wusr[i, j] = 0
                wusp[i, j] = 0
                clvr[i, j] = 0
                clvp[i, j] = 0
                erfL[i, j] = 0
                erfR[i, j] = 0
                ham[i, j] = 0

def apply_boundary_conditions(values, mask):
    """
    境界条件を適用する関数。
    境界外の要素を0に設定し、必要に応じて内側の値をコピーする。
    """
    # 境界外の要素を0に設定
    values[~mask] = 0
    return values

def sol_diffusion(x, y, wusr, wusp, clvr, clvp, erfL, erfR, ham,
                 pull, ID, points_list, layers, divide_active=True):
    dt, dx, dy, a, step, result_interval = pull

    # 初期面積の計算
    point_coords = np.array([[p['x'], p['y']] for p in points_list])
    vor = Voronoi(point_coords)
    initial_areas = calculate_cell_areas(vor, points_list)

    points = points_list  # シードポイントの初期化

    results = {
        'wusr': [], 'wusp': [], 'clvr': [], 'clvp': [], 'erfL': [], 'erfR': [], 'ham': [], 'n': []
    }

    for n in range(step):
        # シードポイントの更新
        points = update_seed_points(points)
        point_coords = np.array([[p['x'], p['y']] for p in points])
        point_layers = [p['layer'] for p in points]

        # ボロノイ分割の再計算
        vor = Voronoi(point_coords)

        # 面積の再計算
        areas = calculate_cell_areas(vor, points)

        # 細胞分裂の実行
        points, initial_areas = divide_cells(points, areas, initial_areas, divide_active=divide_active)

        print(f"  更新後のシードポイント数: {len(points)}")

        # マスクの再作成
        masks = np.zeros((len(points), len(y), len(x)), dtype=np.bool_)
        X, Y = np.meshgrid(x, y)
        for idx, region_idx in enumerate(vor.point_region):
            region = vor.regions[region_idx]
            if -1 in region or len(region) == 0:
                continue

            polygon = vor.vertices[region]
            # バウンディングボックスの計算
            min_x, min_y = np.min(polygon, axis=0)
            max_x, max_y = np.max(polygon, axis=0)
            # バウンディングボックス内のポイントのみチェック
            mask_subset = (X >= min_x) & (X <= max_x) & (Y >= min_y) & (Y <= max_y)
            subset_X = X[mask_subset]
            subset_Y = Y[mask_subset]
            if len(subset_X) == 0:
                continue
            grid_points = np.vstack((subset_X.ravel(), subset_Y.ravel())).T
            mask_cell = Path(polygon).contains_points(grid_points).reshape(-1)
            mask_full = np.zeros(X.shape, dtype=bool)
            mask_full[mask_subset] = mask_cell
            masks[idx] = mask_full

        # メリステム境界マスクの作成
        current_radius = radius_func(n)
        mask = meristem_boundary_mask(x, y, ellip_x, ellip_y, lambda: current_radius)

        # 更新前の値をコピー
        wusr0, wusp0, clvr0, clvp0, erfL0, erfR0, ham0 = (
            wusr.copy(), wusp.copy(), clvr.copy(), clvp.copy(), erfL.copy(), erfR.copy(), ham.copy())

        # 物質の濃度更新
        update_values(wusr, wusp, clvr, clvp, erfL, erfR, ham,
                     wusr0, wusp0, clvr0, clvp0, erfL0, erfR0, ham0,
                     dt, dx, dy, masks, mask,
                     k_Wr, k_WL, K_WL, n_WL, k_WC, K_WC, n_WC, ki_W,
                     K_WW, n_WW, b_Wr, k_Wp, b_Ww, D_Ww,
                     k_Cr, K_CL, n_CL, a_Cc, K_CW, n_CW, K_CH, n_CH,
                     b_Cr, k_Cp, D_Cc, k_Ll, b_Ll, D_Ll, k_pp, b_pp)

        # 境界条件の適用
        wusr = apply_boundary_conditions(wusr, mask)
        wusp = apply_boundary_conditions(wusp, mask)
        clvr = apply_boundary_conditions(clvr, mask)
        clvp = apply_boundary_conditions(clvp, mask)
        erfL = apply_boundary_conditions(erfL, mask)
        erfR = apply_boundary_conditions(erfR, mask)
        ham = apply_boundary_conditions(ham, mask)

        if n % result_interval == 0:
            # 結果を保存
            results['wusr'].append(wusr.copy())
            results['wusp'].append(wusp.copy())
            results['clvr'].append(clvr.copy())
            results['clvp'].append(clvp.copy())
            results['erfL'].append(erfL.copy())
            results['erfR'].append(erfR.copy())
            results['ham'].append(ham.copy())
            results['n'].append(n)

    return results, point_coords, vor, point_layers

def plot_voronoi_with_layers(vor, points, layers, ellip_x, ellip_y, ellip_long, ellip_short):
    """層情報を持つボロノイ図の描画"""
    fig, ax = plt.subplots(figsize=(8, 8))
    voronoi_plot_2d(vor, ax=ax, show_vertices=False, line_colors='gray',
                    line_width=1, line_alpha=0.6, point_size=2)

    # 層ごとにポイントをプロット
    layer_colors = {'L1': 'red', 'L2': 'green', 'L3': 'blue'}
    for layer in ['L1', 'L2', 'L3']:
        idx = [i for i, l in enumerate(layers) if l == layer]
        ax.plot(points[idx, 0], points[idx, 1], 'o', label=layer, color=layer_colors[layer])

    # 楕円の描画(外側の境界線)
    theta = np.linspace(0, 2 * np.pi, 100)
    ellipse_x = ellip_x + ellip_long * np.cos(theta)
    ellipse_y = ellip_y + ellip_short * np.sin(theta)
    #ax.plot(ellipse_x, ellipse_y, 'k--', linewidth=1)

    # L1層とL2層の境界線（オプション）
    # L1層の境界線
    shift_factor_L1 = 0.9  # 縮小率
    shift_factor_L2 = 0.7  # 縮小率
    ellipse_L1_x = ellip_x + ellip_long * shift_factor_L1 * np.cos(theta)
    ellipse_L1_y = ellip_y + ellip_short * shift_factor_L1 * np.sin(theta)
    ax.plot(ellipse_L1_x, ellipse_L1_y, 'r--', linewidth=1)

    # L2層の境界線
    ellipse_L2_x = ellip_x + ellip_long * shift_factor_L2 * np.cos(theta)
    ellipse_L2_y = ellip_y + ellip_short * shift_factor_L2 * np.sin(theta)
    ax.plot(ellipse_L2_x, ellipse_L2_y, 'g--', linewidth=1)

    ax.set_xlim(0, x_size)
    ax.set_ylim(0, y_size)
    ax.legend()
    plt.title('Voronoi Diagram with Layers')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.grid(True)
    plt.show()

def plot(x, y, z, n, name, color, ID, vor=None, points=None, layers=None, ellip_x=None, ellip_y=None, ellip_long=None, ellip_short=None):
    plt.rcParams['font.size'] = 14
    plt.rcParams['xtick.direction'] = 'in'
    plt.rcParams['ytick.direction'] = 'in'

    fig, ax1 = plt.subplots(figsize=(x_size, y_size))
    ax1.yaxis.set_ticks_position('both')
    ax1.xaxis.set_ticks_position('both')
    ax1.set_xlabel('x')
    ax1.set_ylabel('y')

    # メインのデータの描画
    im = ax1.imshow(z.T, vmin=0, vmax=100, extent=[0, x_size, 0, y_size], aspect='equal', cmap=f'{color}', origin='lower')
    ax1.text(0.1, 0.1, f'{name}, Step={n}', color="black")
    ax1.text(7, 0.1, f'{name}, Step={n}', color="white")
    cbar = fig.colorbar(im)
    cbar.set_label(f'{name}')

    # ボロノイ分割とシードポイントの描画
    if vor is not None and points is not None:
        voronoi_plot_2d(vor, ax=ax1, show_vertices=False, line_colors='yellow', line_width=2, line_alpha=0.6, point_size=10)
        # 層ごとにポイントをプロット
        layer_colors = {'L1': 'red', 'L2': 'green', 'L3': 'blue'}
        for layer in ['L1', 'L2', 'L3']:
            idx = [i for i, l in enumerate(layers) if l == layer]
            ax1.plot(points[idx, 0], points[idx, 1], 'o', label=layer, color=layer_colors[layer])

        # 楕円の描画
        theta = np.linspace(0, 2 * np.pi, 100)
        ellipse_x = ellip_x + ellip_long * np.cos(theta)
        ellipse_y = ellip_y + ellip_short * np.sin(theta)
        ax1.plot(ellipse_x, ellipse_y, 'w--', linewidth=2)  # 楕円の境界を白に

    # x軸とy軸の範囲を設定
    ax1.set_xlim(0, x_size)
    ax1.set_ylim(0, y_size)
    ax1.legend()

    # メモリ上に保存
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    img_stock[name].append(Image.open(buf))

    plt.show()
    plt.close(fig)

def plot_diffusion(x, y, results, points, vor, layers, ellip_x, ellip_y, ellip_long, ellip_short, ID):
    n_list = results['n']
    for idx, n in enumerate(n_list):
        element_var = [
            results['wusr'][idx].T, results['wusp'][idx].T, results['clvr'][idx].T, results['clvp'][idx].T, results['erfL'][idx].T, results['erfR'][idx].T, results['ham'][idx].T
        ]

        for field, name, color in zip(
            element_var,
            ["wusr", "wusp", "clvr", "clvp", "erfL", "erfR", "ham"],
            [c_wusr, c_wusp, c_clvr, c_clvp, c_erfL, c_erfR, c_ham]
        ):
            if globals().get(f'plsv_{name}', True):
                plot(
                    x, y, field, n=n, name=name, color=color, ID=ID, vor=vor, points=points, layers=layers,
                    ellip_x=ellip_x, ellip_y=ellip_y,
                    ellip_long=ellip_long, ellip_short=ellip_short
                )

# 初期フィールドの生成
x, y = np.linspace(0, x_max, int(x_max / dx)), np.linspace(0, y_max, int(y_max / dy))
wusr, wusp, clvr, clvp, erfL, erfR, ham = initial_field(x, y)

# 境界条件を設定する
wusr = boundary_condition(wusr)
wusp = boundary_condition(wusp)
clvr = boundary_condition(clvr)
clvp = boundary_condition(clvp)
erfL = boundary_condition(erfL)
erfR = boundary_condition(erfR)
ham = boundary_condition(ham)

# 安定性の確認
nu_x, nu_y = a * dt / dx ** 2, a * dt / dy ** 2
print('nu_x, nu_y=', nu_x, nu_y)

# シードポイントの生成
points_list = seed_points_maker(
    ellip_x=ellip_x,
    ellip_y=ellip_y,
    ellip_long=ellip_long,
    ellip_short=ellip_short,
    num_L1=30,  # L1 層のポイント数（調整可能）
    num_L2=33,  # L2 層のポイント数（調整可能）
    num_L3=40   # L3 層のポイント数（調整可能）
)

# ポイントの座標と属性を取得
points = np.array([[p['x'], p['y']] for p in points_list])
layers = [p['layer'] for p in points_list]

# シミュレーションの実行時に分裂を無効化
divide_active = False  # 一時的に分裂を無効化

pull_sol = [dt, dx, dy, a, step, result_interval]
ID = 0
results, points, vor, layers = sol_diffusion(
    x, y, wusr, wusp, clvr, clvp, erfL, erfR, ham,
    pull_sol, ID, points_list, layers,
    divide_active=divide_active
)

# プロファイリングの実行
pr = cProfile.Profile()
pr.enable()

# 実行する関数
# 注意: ここでは再度 `sol_diffusion` を呼び出さないように修正
# results, points, vor, layers = sol_diffusion(x, y, wusr, wusp, clvr, clvp, erfL, erfR, ham, pull_sol, ID, points_list, layers, divide_active=divide_active)

pr.disable()
s = io.StringIO()
sortby = 'cumulative'
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats(10)  # 上位10個の関数を表示
print(s.getvalue())

# 結果のプロット
plot_diffusion(x, y, results, points, vor, layers, ellip_x, ellip_y, ellip_long, ellip_short, ID)


Output hidden; open in https://colab.research.google.com to view.